## Concatenating different datasets

In [1]:
import pandas as pd

In [2]:
df_original = pd.read_csv("datasets/english.csv", sep=';')

In [3]:
df_eng = pd.read_csv("datasets/CRA_train_1200.csv")

In [4]:
df_deutsch = pd.read_csv("datasets/deutsch.csv", sep=';')

In [5]:
df = pd.concat([df_original, df_eng, df_deutsch], ignore_index=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2400 non-null   float64
 1   pr_txt            3600 non-null   object 
 2   Категория         3600 non-null   object 
 3   Уровень рейтинга  3600 non-null   object 
 4   Id                1200 non-null   float64
dtypes: float64(2), object(3)
memory usage: 140.8+ KB


In [7]:
df = df.drop(columns=['Id', 'Unnamed: 0'])

In [8]:
df

,pr_txt,Категория,Уровень рейтинга
0,Повышение кредитного рейтинга Акционерного общ...,A,A
1,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB
2,"НКР повысила кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A
3,«Эксперт РА» присвоил ПАО «ФосАгро» кредитный ...,AAA,AAA
4,29 марта 2023 Ведущий рейтинговый аналитик Алл...,BBB,BBB
...,...,...,...
3595,«Эксперт РА» подтвердило рейтинг «Татнефти» на...,AAA,AAA
3596,«Эксперт РА» понизило рейтинг компании «Инвест...,BB,BB
3597,АКРА подтвердило кредитный рейтинг ПАО «КАМАЗ»...,A,A+
3598,АКРА подтвердило кредитный рейтинг ФГУП «Почта...,AAA,AAA


## Preprocessing

In [9]:
from nltk.corpus import stopwords
import re
from tqdm.auto import tqdm
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    PER,
    NamesExtractor,
    MorphVocab,
    Doc,
    NewsNERTagger
)

In [10]:
russian_stopwords = stopwords.words('russian')
words_to_del = ['хорошо']
words_to_add = ['rating', 'ооо', 'оао', 'ао', "ра", 'пао', 'гк', 'ппк', 'зао', 'нкр', "акр", "акра", "аналитик", 'компания', 'эксперт','рейтинг', 'агенство', 'млрд', 'руб', 'м', 'х', 'нпк', 'овк', 'г', 'гг', 'i', 'ii']
russian_stopwords.extend(words_to_add)

for w in words_to_del:
    russian_stopwords.remove(w)

In [11]:
def delete_sites(s):
    sites = r"(?:https?:\/\/|ftps?:\/\/|www\.)\S"
    emails='([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)'
    r =  re.sub(sites, '', s)
    return re.sub(emails, '~', r)

In [12]:
import string
def delete_punkt(s):
    return s.translate(str.maketrans('', '', string.punctuation + "«»—№–"))

In [13]:
def delete_digits(s):
    return re.sub("[0-9]", "", s)

In [16]:
def delete_stopwords(tokens, names):
    without_stopwords = [t for t in tokens
                         if t not in russian_stopwords and t not in names]
    return " ".join(without_stopwords)

In [17]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

names_extractor = NamesExtractor(morph_vocab)
ner_tagger = NewsNERTagger(emb)

In [18]:
def normalized(text):
    doc = Doc(text)
    
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    lemmatized_sentence_list = []
    
    doc.tag_ner(ner_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemmatized_sentence_list.append(token.lemma)
        
    names = []
    for span in doc.spans:
        if span.type == PER:
            span.normalize(morph_vocab)
            names.extend(span.normal.lower().split())
            
    text = delete_stopwords(lemmatized_sentence_list, names)
              
    return text

In [19]:
def preprocess(text):
    text_without_sites = delete_sites(text)
    text_without_punct = delete_punkt(text_without_sites)
    text_without_digits = delete_digits(text_without_punct)
    return normalized(text_without_digits)

In [21]:
tqdm.pandas()
df['preprocessed_text'] = df['pr_txt'].progress_apply(preprocess)

  0%|          | 0/3600 [00:00<?, ?it/s]

In [22]:
df.head()

,pr_txt,Категория,Уровень рейтинга,preprocessed_text
0,Повышение кредитного рейтинга Акционерного общ...,A,A,повышение кредитный акционерный общество ураль...
1,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB,подтвердить кредитный энергоконцепт уровень мо...
2,"НКР повысила кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A,повысить кредитный отэкопортсервис стабильный ...
3,«Эксперт РА» присвоил ПАО «ФосАгро» кредитный ...,AAA,AAA,присвоить фосагро кредитный уровень москва мар...
4,29 марта 2023 Ведущий рейтинговый аналитик Алл...,BBB,BBB,март ведущий рейтинговый юров младший директор...


## Making Regression Target

In [23]:
new_target_values = {'AAA': 9,
                     'AA+': 7, 
                     'AA': 6,
                     'AA-': 5,
                     'A+': 4,
                     'A': 3,
                     'A-': 2,
                     'BBB+': 1,
                     'BBB': 0,
                     'BBB-': -1,
                     'BB+': -2,
                     'BB': -3,
                     'BB-': -4,
                     'B+':  -5,
                     'B': -6,
                     'B-': -7,
                     'C': -9}

In [24]:
df['target'] = df['Уровень рейтинга'].apply(lambda x: new_target_values[x])

In [27]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [28]:
scaler = MinMaxScaler(feature_range=(0, 1))

input_vector = np.array(list(range(-10, 11)))  
input_vector = input_vector.reshape(-1, 1)

scaled_vector = scaler.fit_transform(input_vector).reshape(1, -1)[0]

In [29]:
scaled_target = dict(zip(input_vector.reshape(1, -1)[0], scaled_vector))
df['scaled_target'] = df['target'].apply(lambda x: scaled_target[x])

In [38]:
scaled_target

{-10: 0.0,
 -9: 0.04999999999999999,
 -8: 0.09999999999999998,
 -7: 0.14999999999999997,
 -6: 0.19999999999999996,
 -5: 0.25,
 -4: 0.3,
 -3: 0.35,
 -2: 0.4,
 -1: 0.45,
 0: 0.5,
 1: 0.55,
 2: 0.6,
 3: 0.65,
 4: 0.7,
 5: 0.75,
 6: 0.8,
 7: 0.8500000000000001,
 8: 0.9,
 9: 0.95,
 10: 1.0}

----

In [30]:
df

,pr_txt,Категория,Уровень рейтинга,preprocessed_text,target,scaled_target
0,Повышение кредитного рейтинга Акционерного общ...,A,A,повышение кредитный акционерный общество ураль...,3,0.65
1,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB,подтвердить кредитный энергоконцепт уровень мо...,-3,0.35
2,"НКР повысила кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A,повысить кредитный отэкопортсервис стабильный ...,3,0.65
3,«Эксперт РА» присвоил ПАО «ФосАгро» кредитный ...,AAA,AAA,присвоить фосагро кредитный уровень москва мар...,9,0.95
4,29 марта 2023 Ведущий рейтинговый аналитик Алл...,BBB,BBB,март ведущий рейтинговый юров младший директор...,0,0.50
...,...,...,...,...,...,...
3595,«Эксперт РА» подтвердило рейтинг «Татнефти» на...,AAA,AAA,подтвердить татнефть уровень москва апрель рей...,9,0.95
3596,«Эксперт РА» понизило рейтинг компании «Инвест...,BB,BB,понизить уровень изменить прогноз развитие уст...,-3,0.35
3597,АКРА подтвердило кредитный рейтинг ПАО «КАМАЗ»...,A,A+,подтвердить кредитный камаз прогноз стабильный...,4,0.70
3598,АКРА подтвердило кредитный рейтинг ФГУП «Почта...,AAA,AAA,подтвердить кредитный фгуп почта россия уровен...,9,0.95


In [37]:
df.to_csv('datasets/dataset.csv')